# Avaliando o modelo Sentence-BERT com Cosine Similarity Loss

Neste notebook, vamos avaliar o modelo treinado anteriormente, calculando a similaridade entre pares de sentenças do conjunto de teste e comparando com o **ground truth** (dataset "gabarito").

## 1. Instalação e importação

In [6]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy.stats import spearmanr, pearsonr
from sentence_transformers import SentenceTransformer, util

## 2. Carregar o modelo e os dados

In [7]:
MODEL_PATH = '../output/sbert-cosine'
TRAIN_PATH = '../data/train.csv'
TEST_PATH  = '../data/test.csv'
GT_PATH    = '../data/ground_truth.csv'
SUB_PATH   = '../data/submission.csv'

model = SentenceTransformer(MODEL_PATH)
df_train = pd.read_csv(TRAIN_PATH)
df_test  = pd.read_csv(TEST_PATH)
df_gt    = pd.read_csv(GT_PATH)

print(f"🔹 Total de pares de teste: {len(df_test)}")
df_test.head()

🔹 Total de pares de teste: 1208


,id,sentence1,sentence2
0,0,A man is singing and playing a guitar.,A man is playing a guitar.
1,1,Suarez set for Cup comeback,French set for Mali ground combat
2,2,It's not a good idea.,It's a good question.
3,3,Selenski had previously spent seven years in p...,Selenski had served about seven years in priso...
4,4,Russia claims to have foiled possible terror act,Russia says ballistic 'objects' fired in Medit...


## 3. Geração de embeddings e cálculo do cosseno na parte de calibração

Aprende uma calibração linear (coeficientes a e b) que ajusta a escala da similaridade do cosseno para o intervalo real das anotações (0–5), usando dados rotulados.
Depois aplica essa transformação aos exemplos de teste sem rótulo, sem vazamento de informação.

In [8]:
bins = pd.cut(df_train['similarity_score'], bins=[0,1,2,3,4,5], labels=False, include_lowest=True)
train_rest, calib = train_test_split(
    df_train, test_size=0.2, random_state=42, stratify=bins
)

emb1_c = model.encode(calib['sentence1'].tolist(), convert_to_tensor=True)
emb2_c = model.encode(calib['sentence2'].tolist(), convert_to_tensor=True)
cos_c  = util.cos_sim(emb1_c, emb2_c).diagonal().cpu().numpy()  # [-1,1]
y_c    = calib['similarity_score'].astype(float).values          # 0–5

Xc = np.c_[np.ones_like(cos_c), cos_c]
(a, b), *_ = np.linalg.lstsq(Xc, y_c, rcond=None)
print(f"Calibração:  score ≈ {a:.4f} + {b:.4f}·cos")

Calibração:  score ≈ 0.0200 + 4.7090·cos


## 4. Calcular a similaridade entre embeddings

Aqui o modelo transforma cada sentença em um vetor numérico **(embedding)** e calcula o quão parecidos esses vetores são usando a **Cosine Similarity**.
Depois, a calibração aprendida anteriormente é usada para ajustar o valor do cosseno para a escala original (0–5), gerando a previsão final de similaridade.

In [9]:
emb1_t = model.encode(df_test['sentence1'].tolist(), convert_to_tensor=True)
emb2_t = model.encode(df_test['sentence2'].tolist(), convert_to_tensor=True)
cos_t  = util.cos_sim(emb1_t, emb2_t).diagonal().cpu().numpy()

pred_test = a + b * cos_t
pred_test = np.clip(pred_test, 0.0, 5.0)
df_test['predictions'] = pred_test

## 5. Salvar previsões

In [10]:
df_test[['id', 'predictions']].to_csv('../data/submission.csv', index=False)
print('✅ Previsões salvas em: ../data/submission.csv')

✅ Previsões salvas em: ../data/submission.csv


## 6. Avaliar com o Ground Truth

"Ground Truth" é um dataset com as predições corretas, semelhante a um "gabarito" do conjunto de teste.

In [ ]:
sub = pd.read_csv(SUB_PATH)
merged = sub.merge(df_gt, on='id', suffixes=('_pred','_real'))
y_pred = merged['predictions_pred'].to_numpy(dtype=float)
y_true = merged['predictions_real'].to_numpy(dtype=float)

mse = mean_squared_error(y_true, y_pred)
print(f"> MSE: {mse:.4f}")

if mse < 0.5:
    print("✅ Excelente (erro médio < ~0.5).")
elif mse < 1.5:
    print("🟡 Bom — dá para melhorar com mais treino/calibração.")
elif mse < 3.0:
    print("⚠️ Razoável — modelo ainda concentrando no centro.")
else:
    print("❌ Ruim — verifique dados/escala/calibração.")

> MSE: 0.4996
✅ Excelente (erro médio < ~0.7).


## Conclusão

- O modelo **Sentence-BERT** foi treinado com **Cosine Similarity Loss**, aprendendo a gerar embeddings que preservam a proximidade semântica entre pares de sentenças.
- Após o treinamento, foi aplicada uma **calibração linear sem vazamento**, ajustando a escala das similaridades de cosseno ([-1, 1]) para a escala real das anotações (0 a 5).
- Esse tipo de abordagem pode ser aplicado em diversas tarefas práticas, como:
    - Detecção de plágio e redundância textual;
    - Busca semântica e recuperação de informações;
    - Sistemas de recomendação de perguntas, respostas ou documentos;
    - Análise de coerência e duplicidade em textos longos.